# Obtener información requerida para el Modelo DuPont

In [ ]:
# Requerimientos
!pip install selenium pandas sqlalchemy

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import sqlalchemy

db = sqlalchemy.create_engine("sqlite:///data.sqlite")

In [5]:
def get_raws(country):
    df = pd.read_sql(f"select * from {country}_companies", db)
    raws = []
    for row in df.values:
        source = row[5] 
        
        raws.append({
            "source": source,
            "revenue": f"{source}financials-income-statement/total-revenue/",
            "income": f"{source}financials-income-statement/net-income/",
            "assets": f"{source}financials-balance-sheet/total-assets/",
            "equity": f"{source}financials-balance-sheet/shrhldrs-equity/",
            "status": 0
        })
    pd.DataFrame(raws).to_sql(f"{country}_raws", db, if_exists="replace")

for country in ["chile", "colombia", "mexico", "peru"]:
    get_raws(country)

In [2]:
from sqlalchemy.sql import text

def get_by(url):
    try:
        driver = webdriver.Edge()
        driver.get(url)
        div = driver.find_element(By.CLASS_NAME, "table-CbBHHTvu")
        return div.text
    except:
        return url

def get_info(country):
    df = pd.read_sql(f"select * from {country}_raws", db)

    for row in df.values:
        if row[6] == 0:
            smt = text(f"""
                UPDATE {country}_raws SET
                    revenue=:revenue, income=:income, assets=:assets, equity=:equity, status=1
                WHERE source=:source
            """)
            params = {
                "source": row[1],
                "revenue": get_by(row[2]),
                "income": get_by(row[3]),
                "assets": get_by(row[4]),
                "equity": get_by(row[5])
            }
            with db.begin() as conn:
                conn.execute(smt, params)
                conn.commit()

for country in ["chile", "colombia", "mexico", "peru"]:
    get_info(country)

In [ ]:
df = pd.read_sql("select * from peru_raws", db)
df.tail()